# Basic ML Model Deployment

## Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.preprocessing import StandardScaler
import pickle
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

## Fetch Data

In [2]:
#data=pd.read_csv('https://raw.githubusercontent.com/tkseneee/Dataset/master/Loan_data_ver2.csv')
data=pd.read_csv('loan_dataset.csv',
                 dtype  = {'Married': str,
                     'Education': str,
                     'ApplicantIncome': 'Int64',
                     'LoanAmount': 'Int64',
                     'Credit_History': 'Int64',} )


## Explore Data

In [3]:
data.shape

(614, 6)

In [4]:
data.columns

Index(['Married', 'Education', 'ApplicantIncome', 'LoanAmount',
       'Credit_History', 'Loan_Status'],
      dtype='object')

In [5]:
data.dtypes

Married             object
Education           object
ApplicantIncome      Int64
LoanAmount           Int64
Credit_History       Int64
Loan_Status        float64
dtype: object

In [6]:
data.head(2)

,Married,Education,ApplicantIncome,LoanAmount,Credit_History,Loan_Status
0,No,Graduate,5849,<NA>,1,0.10
1,Yes,Graduate,4583,128,1,0.32


In [7]:
# fetch features with missing values
data.isnull().sum()

Married             3
Education           0
ApplicantIncome     0
LoanAmount         22
Credit_History     50
Loan_Status         0
dtype: int64

3 features namely - Married,LoanAmount,Credit_History has missing values

In [8]:
data['Married'].value_counts()


Married
Yes    398
No     213
Name: count, dtype: int64

In [9]:
data['Education'].value_counts()

Education
Graduate        449
Not Graduate    127
HSC              38
Name: count, dtype: int64

In [10]:
# segreegating target & feature
X=data.drop('Loan_Status', axis=1)
y=data['Loan_Status']

In [11]:
# spliting data into train & validation set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=48)

In [12]:
# fetching numeric features list
feat_num=list(X.select_dtypes(include=np.number).columns)
feat_num

['ApplicantIncome', 'LoanAmount', 'Credit_History']

In [13]:
# fetching categorical features  list
feat_cat=list(X.select_dtypes(exclude=np.number).columns)
feat_cat

['Married', 'Education']

## Defining Data processing & Modeling  Pipeline

In [14]:
num_pipe=Pipeline([('imputer',SimpleImputer(strategy='mean')),('std_scale',StandardScaler())])

num_pipe

Pipeline(steps=[('imputer', SimpleImputer()), ('std_scale', StandardScaler())])

In [15]:
# pipeline for categorical faetures - missing category replacement by new category i.e. missing followed by one hot encoding 
feat_pipe = Pipeline([('imputer',SimpleImputer(strategy='constant', fill_value='Missing')), 
                      ('one_hot',(OneHotEncoder()))]) 

feat_pipe

Pipeline(steps=[('imputer',
                 SimpleImputer(fill_value='Missing', strategy='constant')),
                ('one_hot', OneHotEncoder())])

In [16]:
#combine data processing pipeline
data_pipeline=ColumnTransformer([('numeric',num_pipe,feat_num),
                                 ('categorical',feat_pipe, feat_cat)],
                                  remainder='passthrough')




data_pipeline

ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('std_scale',
                                                  StandardScaler())]),
                                 ['ApplicantIncome', 'LoanAmount',
                                  'Credit_History']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Missing',
                                                                strategy='constant')),
                                                 ('one_hot', OneHotEncoder())]),
                                 ['Married', 'Education'])])

In [17]:
# adding ml-model into pipeline 
full_pipe=Pipeline([('pre_process',data_pipeline),('model',RandomForestRegressor())])

In [18]:
# training
full_pipe.fit(X_train,y_train)

Pipeline(steps=[('pre_process',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('std_scale',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'LoanAmount',
                                                   'Credit_History']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot',
                                                                   OneHotEncoder())]),
                                                  ['Married', 'Education'])])),
                ('model', RandomForestRegressor())])

In [19]:
# prediction
full_pipe.predict(X_test)

array([0.1649, 0.198 , 0.4733, 0.4341, 0.1785, 0.0914, 0.9625, 0.255 ,
       0.2191, 0.3033, 0.2239, 0.9781, 0.2901, 0.1082, 0.0397, 0.1182,
       0.1248, 0.0717, 0.0393, 0.2089, 0.2965, 0.089 , 0.3229, 0.4363,
       0.5432, 0.128 , 0.2628, 0.2   , 0.2751, 0.0129, 0.1048, 0.2182,
       0.4058, 0.9456, 0.1304, 0.2727, 0.2041, 0.5826, 0.9589, 0.1892,
       0.2861, 0.1599, 0.2972, 0.349 , 0.4023, 0.434 , 0.1233, 0.1724,
       0.2386, 0.1996, 0.1242, 0.394 , 0.081 , 0.1222, 0.9638, 0.0675,
       0.4285, 0.2586, 0.2578, 0.1967, 0.3004, 0.7177, 0.2676, 0.1727,
       0.4682, 0.2448, 0.0753, 0.2887, 0.1865, 0.2142, 0.4329, 0.0969,
       0.2946, 0.3379, 0.0714, 0.804 , 0.1083, 0.4071, 0.5717, 0.0454,
       0.2539, 0.096 , 0.3243, 0.7945, 0.4305, 0.1555, 0.1029, 0.15  ,
       0.0466, 0.497 , 0.0462, 0.2177, 0.2427, 0.549 , 0.4187, 0.1858,
       0.9714, 0.1129, 0.0775, 0.3371, 0.5279, 0.0816, 0.0063, 0.0528,
       0.0812, 0.125 , 0.3259, 0.2356, 0.0247, 0.1193, 0.1505, 0.0892,
      

In [20]:
## can store numeric and categorical variables also as pickle file
# pickle.dump(feat_num,open('feat_numv1','wb'))
# pickle.dump(feat_cat,open('feat_catv1','wb'))

 

## Store the model as pickle file 

In [21]:
#pickle.dump(full_pipe,open('full_pipeline.pkl','wb'))
joblib.dump(full_pipe, 'full_pipeline.pkl')

['full_pipeline.pkl']